In [26]:
from lib.util import getCodes, displayMarkdown, getCitations, getCodeset
from IPython.core.display import HTML, Markdown, Latex

import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import re, yaml
import vega_datasets

codeset = pd.read_csv('data/codeset.csv', ).replace(np.nan, '', regex=True)

In [27]:
# Import codes
#
# The cell below recursively searches the `notebooks/` directory for files with a `.html.pdf` extension. These 
# are PDF printouts of computational notebooks and scripts that I coded using the comments feature in 
# Adobe Acrobat DC (https://acrobat.adobe.com/ca/en/acrobat.html). This notebook needs to import all 
# codes from the open coding process.
codes = pd.read_csv('data/code-analysis-network.csv')

In [28]:
# Import metadata on analyzed notebooks
# 
# The cell below parses the works-cited section of the `README.md` document with regular expressions. 
# A demo of this regular expression can be found on [regexr.com](https://regexr.com/4htcn). 
# This section contains a citation for every notebook included in this analysis. 
# Some of the URLs from the works cited list are cleaned so that the path matches the path in this repository 
# under `notebooks/`.

citations = getCitations()

In [29]:
# Import repos and contributors
#
# The file `notebook-search.ipynb` mines GitHub for repos containing data-journalism 
# workflows and exports this data to `data/repos.csv` and `data/contributors.csv`.

# Get all repos
repos = pd.read_csv('data/repos.csv')

# Subset repos to only those that contained data analysis


# Get contributors, which is really contributors to keeper notebooks
contributors = pd.read_csv('data/contributors.csv')

In [30]:
# Notebook coding order
# 
# The works cited page of analyzed computational notebooks 
# serves as a list of coded notebooks, but it doesn't preserve the order 
# in which they were coded. The `reposIncluded` list has lists each repo in 
# reverse chronological order.

pathsIncluded = [
    'demolitions',
    'gunsales',
    'us-weather-history',
    'california-buildings-in-severe-fire-hazard-zones',
    'swana-population-map',
    '1805-regionen im fokus des US-praesidenten',
    'school-choice',
    '201901-achievementgap',
    'general-election-2015-classification-tree',
    '201901-hospitalquality',
    'awb-notebook',
    'skatemusic',
    'new-york-schools-assessment',
    'lending-club',
    'auditData',
    '2019-ems-analysis',
    'federal_employees_trump_2017', 
    'infrastructure-jobs',
    'librarians',
    'midwife-led-units',
    'internal-migration-london',
    'electric-car-charging-points',
    'school-star-ratings-2018',
    'prison-admissions',
    'vox-central-line-infections',
    'verge-uber-launch-dates',
    'buster-posey-mvp',
    'work-from-home',
    'nyc-trips',
    'bob-ross',
    'bechdel',
    'employment-discrimination',
    '2015-11-refugees-in-the-united-states',
    'babyname_politics',
    'wikipedia-rankings',
    'Power_of_Irma',
    'Endangered-Species-Act-Louisiana',
    'california-h2a-visas-analysis',
    '2016-04-republican-donor-movements',
    'the-cube-root-law',
    '2016-09-shy-trumpers',
    '2018-05-31-crime-and-heat-analysis',
    '2016-11-bellwether-counties',
    'heat-index',
    '2018-voter-registration',
    'long-term-care-db',
    'census-hard-to-map-analysis',
    'california-crop-production-wages-analysis',
    'california-ccscore-analysis',
    '2019-04-democratic-candidate-codonors',
]

codingOrder = pd.DataFrame({'analysis': pathsIncluded[::-1] }).reset_index().rename(columns={'index': 'coding order'})

In [31]:
# Sanity check
#
# Make sure that I've manually added all the repos to `reposIncluded` that are listed in `README.md`.

citMinusPath = set(citations.path.unique()).difference(set(pathsIncluded))
pathMinusCit = set(pathsIncluded).difference(set(citations.path.unique()))

if len(citMinusPath) > 0:
    raise RuntimeError('In `citations` but not in `pathsIncluded`: {}'.format(', '.join(list(citMinusPath))))

if len(pathMinusCit) > 0:
    raise RuntimeError('In `pathsIncluded` but not in `citations`: {}'.format(', '.join(list(pathMinusCit))))

# Establishing Saturation

We establish saturation in our codeset by monitoring the number of unique codes with respect to the number of repos included in our technical observation study. Approaching 50 repos we notice the size of the codeset leveling off. At this point, we determine the codeset has reached saturation, adequately describing data wrangling actions and processes in this domain.

In [37]:
# In order to visualize the cardinality of the code set per notebook coded, 
# the `codes` data frame needs to be grouped by article.

codesByArticle = codes.groupby(['analysis']).name \
    .unique() \
    .to_frame('codes') \
    .reset_index() \
    .merge(codingOrder, on='analysis') \
    .sort_values('coding order') \
    .reset_index()

codesByArticle['cumulative count'] = 0

codeset = set()
for i, row in codesByArticle.iterrows():
    setDiff = set(row.codes).difference(codeset)
    codeset = codeset.union(setDiff)
    codesByArticle.loc[i, 'cumulative count'] = len(codeset)
    codesByArticle.loc[i, 'new codes'] = ', '.join(setDiff)
    
# Plot the size of the code set as more computational notebooks are analyzed.

chart = alt.Chart(data = codesByArticle, title = 'Code Set Growth') \
    .mark_line() \
    .encode(
        x = alt.X('coding order:Q', title="Repos Coded"),
        y = alt.Y('cumulative count:Q', title="Total Unique Codes")
    )

# chart
# chart.save('codeset_growth.png')
# alt.renderers.enable('altair_saver', ['vega-lite', 'png'])

# from altair_saver import save
# save(chart, 'codeset_growth.png')

Markdown('![](figs/codeset_growth.png)')

![](figs/codeset_growth.png)

**Fig. S1**: Codeset growth per repo coded. For each notebook included in analysis list which codes were introduced to the code set. After 23 notebooks, some computational notebooks didn't add any new codes. By 50 notebooks, code set growth was so minimal that we declared our code set converged.

## Newly introduced codes by repo

Below we explicitly list which repos defined new open codes in our codeset. Repos are ordered by when they were coded in our technical observation study.

In [8]:
for i, row in codesByArticle.iterrows():
    displayMarkdown('{}. **{}**: {}'.format(i + 1, row.analysis, row['new codes']))

1. **2019-04-democratic-candidate-codonors**: create child table, trim by categorical value, repetitive code, count unique values, figure a rate, create annotations, outer join, compare groups, deduplicate, create soft key, group by variable, create a frequency table, trim by quantitative threshold, gather, load, format values, export, canonicalize variable names, remove variables, peek at data, govt data portal, union datasets, sort, change var type, self join dataset, aggregate, standardize categorical variables, construct a subroutine, align variables

2. **california-ccscore-analysis**: describe statistically, show trend over time, remove incomplete data, visualize data, calculate spread, count number of rows, standardize variable, trim by date range, inspect data schema, identify extreme values, cross tabulate, divide & conquer, calculate change over time, trim fat

3. **california-crop-production-wages-analysis**: adjust for inflation, construct data manually, construct data pipeline, wrangle data for graphics, combine periodic data, trim by geographic area, inner join, lookup table values

4. **census-hard-to-map-analysis**: parse variable, tolerate dirty data

5. **long-term-care-db**: generate high-level summary, generate dataset identification, scrape web for data, create lookup table, refine table, fill in na values after an outer join, count the data, combine categorical values, edit values, replace na values, use non-public, provided data

...

46. **swana-population-map**: 

47. **california-buildings-in-severe-fire-hazard-zones**: search for clusters

48. **us-weather-history**: validate data quality with domain-specific rules

49. **gunsales**: adjust for season

50. **demolitions**: 

# Incorporating diversity

In order to prevent this code set from being biased by one individual or organization's data wrangling behavior, we deliberately sought out notebooks from a variety of news organizations and data journalists. This analysis comes from, but is not limited to, news organizations that constitute "major players" in data journalism. 

In [9]:
def displayCoverage(df, title, metric, xlab, ylab, ykind='organizations'):
    """"""
    rank = '{}_rank'.format(metric)
    count = '{}_count'.format(metric)
    
    bars = alt.Chart(data = df.sort_values(rank).head(50), title = title) \
        .mark_bar() \
        .encode(
            x = alt.X('{}:Q'.format(count), axis = alt.Axis(title = xlab)),
            y = alt.Y('name:N',
                  sort = alt.EncodingSortField(field = count, order = 'descending'),
                  axis = alt.Axis(title = ylab)
            ),
            color = alt.Color('is_included:N', title = 'Included in analysis?')
        )

    display(bars.properties(height=500, width=500))

## Prolificness of news organizations

Some news organizations are more engaged in data journalism than others. In order for the result of our technical observation study to be representative of the practices of a variety of organizations, we deliberately selected notebooks for inclusion in our technical observation study by news organizations across the spectrum of prolificness in this genre of journalism.

We ranked these organizations by two metrics based on our pool of journalistic code repositories containing data analysis: 

* The count of individual code repositories 
* The number of commits by journalists working for different news organizations

In [10]:
orgs = repos[repos.is_keeper == True] \
    .groupby('org') \
    .agg({
        'url': 'nunique',
        'commits': 'sum'
    }) \
    .reset_index() \
    .rename(columns={
        'org': 'name', 
        'url': 'repo_count',
        'commits': 'commit_count'
    })

# Assign ranks to each organization per repo count and commit count
def rank(df, sort_col, rank_col):
    df.sort_values(sort_col, ascending=False, inplace=True)
    df[rank_col] = df.reset_index().index + 1

rank(orgs, 'repo_count', 'repo_rank')
rank(orgs, 'commit_count', 'commit_rank')

# Find which organizations have been included in analysis
includedOrgs = list(citations.organization.unique())
orgs['is_included'] = False
orgs.loc[orgs.name.isin(includedOrgs), 'is_included'] = True

# Ok, so what did I do?
# orgs.head()

### By number of repos

Most news organizations, including *BuzzFeed News*, *Los Angeles Times*, and the *Austin American-Statesman*, create one repository per analysis work flow. We include at least one repository from the top 19 news organizations by the number of unique repositories in our pool journalistic code repositories containing data analysis. We also deliberately select repositories from news organization that only have one repository in this pool.

In [20]:
# displayCoverage(orgs,
#                 title = 'News Organizations by Repository Counts', 
#                 metric = 'repo',
#                 xlab = 'Number of Repos',
#                 ylab = 'News Organization')
Markdown("![](figs/repo-counts-news-orgs.png)")

![](figs/repo-counts-news-orgs.png)

**Fig. S2**: Repository count by news organization. This bar chart show the number of repositories per news organization in our curated pool of journalistic, data-analysis repositories, color-coded by whether at least one repository from that news organization was included in our technical observation study. Orange values indicate the news organization was included and blue indicates otherwise.

### By commits

However, one limitation of ranking news organizations by the number of repositories that some organizations, such as *FiveThirtyEight* keep computational notebooks for multiple data journalism articles in one master code repository. A *commit* in Git can be thought of as a unit of change. Thus, the more a repository has changed overtime, the more commits. If a news organization is only using one repository for all their data journalism work, then it should have lots of commits.

When ranking news organizations by commit counts, our qualitative analysis includes include the top 18 news organizations by commit count in addition to news organizations with only a few commits in our pool of journalistic code repositories containing data analysis.

In [12]:
# displayCoverage(orgs,
#                 title = 'News Organizations by Repositories Commits', 
#                 metric = 'commit',
#                 xlab = 'Number of Commits',
#                 ylab = 'News Organization')
Markdown('![](figs/repo-commits-news-orgs.png)')

![](figs/repo-commits-news-orgs.png)

**Fig. S3**: News organizations ranked by number of commits. This bar chart show the number of commits per users associated with various news organization in our curated pool of journalistic, data-analysis repositories. The chart is color-coded by whether at least one repository from that news organization was included in our technical observation study. Orange values indicate the news organization was included and blue indicates otherwise.

In [21]:
# Which organizations were included?

included_count = sum(orgs.is_included)
total_orgs = len(orgs.name.unique())

displayMarkdown("""
This analysis includes {included_count} news organizations out of {total} that had computational notebooks deemed relevant to this analysis ({percent}%).
""".format(**{
    'included_count': included_count,
    'total': total_orgs,
    'percent': round((included_count / total_orgs) * 100, 2),
}))

displayMarkdown("""
| Organization | Is included? |
| ------------ | ------------ |
{rows}
""".format(**{
    'rows': '\n '.join([ '| {} | {} |'.format(row[0], 'Yes' if row[5] else 'No') for i, row in orgs.sort_values('name').iterrows() ])
}))


This analysis includes 25 news organizations out of 37 that had computational notebooks deemed relevant to this analysis (67.57%).



| Organization | Is included? |
| ------------ | ------------ |
| Austin American-Statesman | Yes |
 | Australian Broadcasting Corporation | No |
 | BBC | Yes |
 | Baltimore Sun | Yes |
 | BuzzFeed News | Yes |
 | CORRECTIV | Yes |
 | Center for Public Integrity | Yes |
 | Chicago Tribune | No |
 | DataMade | No |
 | Datastory | No |
 | FiveThirtyEight | Yes |
 | Los Angeles Times | Yes |
 | NOLA | Yes |
 | National Public Radio | Yes |
 | Neue Zürcher Zeitung | Yes |
 | New York Times | Yes |
 | Politico | No |
 | Polygraph | Yes |
 | ProPublica | Yes |
 | Quartz | Yes |
 | South Florida Sun Sentinel | No |
 | St Louis Public Radio | Yes |
 | Star Tribune | Yes |
 | Süddeutsche Zeitung | No |
 | Tampa Bay Times | No |
 | The Atlantic | No |
 | The Buffalo News | Yes |
 | The Economist | No |
 | The Oregonian | Yes |
 | The Texas Tribune | No |
 | The Times and Sunday Times | Yes |
 | The Washington Post | Yes |
 | Time | Yes |
 | TrendCT | Yes |
 | Vox | Yes |
 | WBEZ Chicago | No |
 | WUFT | Yes |


## Prolificness of individual journalists

In addition to taking steps to incorporate comprehensiveness and diversity of news organization into our descriptive taxonomy, we also attempt to add comprehensiveness and diversity in the individual journalists.

We exclude some data journalist with commits from this summary because their commits were insignificant contributions to repos such as comments, README file updates, initial repo setup, and general code clean up.
    
* [Andrei Scheinkman](https://github.com/fivethirtyeight/data/commits?author=ascheink), *FiveThirtyEight*

* [Dhrumil Mehta](https://github.com/fivethirtyeight/data/commits?author=dmil), *FiveThirtyEight*
    
* [Stephen Turner](https://github.com/fivethirtyeight/data/commits?author=stephenturner), *FiveThirtyEight*

* [Nate Silver](https://github.com/fivethirtyeight/data/commits?author=natesilver538), *FiveThirtyEight*

* [Dan Nguyen](https://github.com/TheUpshot/leo-senate-model/commits?author=dannguyen), *The Upshot*

* [Derek Willis](https://github.com/BuzzFeedNews/2014-09-rising-sunday-show-guests/commit/780b808606d05a7d79dc6c40e5c64d03a490fe65), *BuzzFeed News*

Note that this summary also excludes journalists who:

* Worked collaboratively and only one of them committed code.
    * Matt Stevens
    * Adam Pearce

* Only were included in the technical observations study via Observable notebooks
    * Sahil Chinoy

* Did not commit their own code. For example, *FiveThirtyEight* code appears to be committed by someone else.
    * Rob Arthur
    * Stefano Ceccon
    * Walt Hickey


In [22]:
# The first step is to split (a.k.a explode) rows in the `citations` data frame that represent collaborative 
# data journalism projects done by multiple journalists into separate rows. I've separated multiple authors 
# in the citations with semicolons. It's important to split the name on semicolon and space when defining 
# `citationsUnpacked` to prevent duplicates.

citationsUnpacked = pd.DataFrame(list(citations.journalist.apply(lambda name: name.split('; '))),
            index=citations.path).stack().to_frame().reset_index([0, 'path']) \
    .rename(columns={0: 'journalist'})

citationJournalists = pd.merge(citationsUnpacked, citations.drop(['journalist'], axis=1), on='path', how='left')
citationJournalists['name'] = citationJournalists.journalist.apply(lambda name: "{} {}".format(*name.split(', ')[::-1]))
citationJournalists['is_cited'] = True
citationJournalists = citationJournalists[['name', 'is_cited']].drop_duplicates()

In [23]:
# Modify the `citations` data frame so that it has a primary key to join the `contributors` data frame. 
# The `journalist` column in `citations` is formatted as "given name, surname", so I'll swap those around.

# From the `contributors` data frame, we can get a sense of the "top" data journalists based on how many 
# commits they've made and the number of followers of their GitHub user account.

# Fun fact, this is an example of aggregate join
dataJournalists = pd.merge(
    contributors[['login', 'name']],
    contributors.groupby('login').agg({'followers': max, 'commits': sum }).reset_index(),
    on='login') \
    .drop_duplicates()

createPk = lambda df: df.name.str.lower().str.replace(r'\s', '')

dataJournalists['pk'] = createPk(dataJournalists)
citationJournalists['pk'] = createPk(citationJournalists)

# Combine to get a data frame of journalists included and not included in analysis
dataJournalists = pd.merge(dataJournalists, citationJournalists, how='left', on='pk') \
    .rename(columns={
        'is_cited': 'is_included',
        'followers': 'follower_count',
        'commits': 'commit_count',
        'name_x': 'name'
    }) \
    .drop(['name_y', 'pk'], axis=1)

dataJournalists.is_included.fillna(False, inplace=True)

# # Rank data journalists by followers and commits
rank(dataJournalists, 'follower_count', 'follower_rank')
rank(dataJournalists, 'commit_count', 'commit_rank')

# # Remove NAs
dataJournalists.dropna(inplace=True)

# Remove duplicates 
dataJournalists.drop_duplicates('login', inplace=True)

#dataJournalists.head()

In [24]:
# Sanity check
#
# Did we loose any journalists from the citations list when we merged it with the list of data journalists? 
# Which journalists were cited that are not in the `dataJournalists` data frame?

# missing = displayMarkdown(', '.join(list(set(citationJournalists.name.str.lower()).difference(set(dataJournalists.name.str.lower())))))

### By commits

In [25]:
blacklist = ['Dhrumil Mehta', 'Derek Willis', 'Stephen Turner', 'Dan Nguyen', 'Nate Silver', 'Andrei Scheinkman']

# displayCoverage(
#     df = dataJournalists[~dataJournalists.name.isin(blacklist)],
#     title = 'Data Journalists Ranked by Commits',
#     metric = 'commit',
#     xlab = 'Commit Count',
#     ylab = 'Name',
#     ykind = 'people'
# )
Markdown('![](figs/data-journalists-commits.png)')

![](figs/data-journalists-commits.png)

**Fig. S4**: Data journalists who authored code repositories in our pool of journalistic, data-analysis repos, ranked by number of commits. This chart is color-coded orange to indicate that the individual authored an analysis included in our technical observation study.

### By followers

Our qualitative analysis is based on repositories authored by the top eight data journalists ranked by the number of followers in addition to many GitHub users with less followers.

In [18]:
# displayCoverage(
#     df = dataJournalists[~dataJournalists.name.isin(blacklist)],
#     title = 'Data Journalists Ranked by Followers',
#     metric = 'follower',
#     xlab = 'Follower Count',
#     ylab = 'Name',
#     ykind = 'people'
# )
Markdown('![](figs/data-journalists-followers.png)')

![](figs/data-journalists-followers.png)

**Figure S5**: Data journalists who authored code repositories in our pool of journalistic, data-analysis repos, ranked by number of followers on GitHub. This chart is color-coded orange to indicate that the individual authored an analysis included in our technical observation study.

# Descriptive cross-check of multi-table framework

We cross check the descriptive power of our multi-table framework for data wrangling by comparing against the high-level axial codes in our descriptive action taxonomy. We only include actions codes that correspond with table operations, hence excluding codes in the Profile branch.

In [19]:
# See https://docs.google.com/spreadsheets/d/17XwOJo8EsbTjDwmfrnWdWvDDzeht6YtUNc-PQLSIFtk/edit?usp=sharing
# for original chart

![cross-check-figure](figs/cross-check.png)